In [5]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        X_train_main = np.load('X_mnist.npy')
        X_train = np.vstack([X_train_main,np.load('aug_x.npy')])
        X_train = np.reshape(X_train, (X_train.shape[0], 28, 28))

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=8001, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\Duicane\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.185699, acc.: 46.88%] [G loss: 0.815582]
1 [D loss: 0.468853, acc.: 76.56%] [G loss: 0.903310]
2 [D loss: 0.311578, acc.: 84.38%] [G loss: 1.002194]
3 [D loss: 0.259861, acc.: 92.19%] [G loss: 1.102615]
4 [D loss: 0.211994, acc.: 95.31%] [G loss: 1.200154]
5 [D loss: 0.223140, acc.: 96.88%] [G loss: 1.290488]
6 [D loss: 0.185164, acc.: 100.00%] [G loss: 1.403405]
7 [D loss: 0.160568, acc.: 100.00%] [G loss: 1.493110]
8 [D loss: 0.145129, acc.: 100.00%] [G loss: 1.608480]
9 [D loss: 0.141134, acc.: 100.00%] [G loss: 1.696235]
10 [D loss: 0.124963, acc.: 100.00%] [G loss: 1.731412]
11 [D loss: 0.099526, acc.: 100.00%] [G loss: 1.932422]
12 [D loss: 0.103253, acc.: 100.00%] [G loss: 1.983094]
13 [D loss: 0.088438, acc.: 100.00%] [G loss: 2.036422]
14 [D loss: 0.082127, acc.: 100.00%] [G loss: 2.057764]
15 [D loss: 0.080407, acc.: 100.00%] [G loss: 2.208255]
16 [D loss: 0.092504, acc.: 100.00%] [G loss: 2.310751]
17 [D loss: 0.071926, acc.: 100.00%] [G loss: 2.306736]
18 [D lo

150 [D loss: 0.994318, acc.: 51.56%] [G loss: 1.713709]
151 [D loss: 0.796072, acc.: 76.56%] [G loss: 1.516941]
152 [D loss: 0.451284, acc.: 79.69%] [G loss: 2.777874]
153 [D loss: 0.103427, acc.: 96.88%] [G loss: 3.659435]
154 [D loss: 0.099797, acc.: 96.88%] [G loss: 4.064971]
155 [D loss: 0.042502, acc.: 100.00%] [G loss: 4.137631]
156 [D loss: 0.105176, acc.: 96.88%] [G loss: 3.902685]
157 [D loss: 0.079418, acc.: 95.31%] [G loss: 3.859665]
158 [D loss: 0.082487, acc.: 100.00%] [G loss: 3.741584]
159 [D loss: 0.130870, acc.: 96.88%] [G loss: 3.828921]
160 [D loss: 0.422921, acc.: 78.12%] [G loss: 2.868581]
161 [D loss: 0.087768, acc.: 96.88%] [G loss: 3.510925]
162 [D loss: 0.158517, acc.: 93.75%] [G loss: 3.719246]
163 [D loss: 0.203161, acc.: 93.75%] [G loss: 2.456529]
164 [D loss: 0.230908, acc.: 89.06%] [G loss: 3.536177]
165 [D loss: 0.157352, acc.: 93.75%] [G loss: 3.820485]
166 [D loss: 0.145615, acc.: 95.31%] [G loss: 3.152598]
167 [D loss: 0.181661, acc.: 92.19%] [G loss: 

302 [D loss: 0.722803, acc.: 46.88%] [G loss: 0.686026]
303 [D loss: 0.694726, acc.: 46.88%] [G loss: 0.698702]
304 [D loss: 0.701554, acc.: 48.44%] [G loss: 0.704301]
305 [D loss: 0.679502, acc.: 43.75%] [G loss: 0.741755]
306 [D loss: 0.682625, acc.: 45.31%] [G loss: 0.753595]
307 [D loss: 0.684543, acc.: 48.44%] [G loss: 0.735143]
308 [D loss: 0.679531, acc.: 56.25%] [G loss: 0.749671]
309 [D loss: 0.698537, acc.: 40.62%] [G loss: 0.718731]
310 [D loss: 0.709355, acc.: 46.88%] [G loss: 0.719742]
311 [D loss: 0.742177, acc.: 39.06%] [G loss: 0.673870]
312 [D loss: 0.670396, acc.: 53.12%] [G loss: 0.720877]
313 [D loss: 0.690154, acc.: 50.00%] [G loss: 0.737103]
314 [D loss: 0.691818, acc.: 43.75%] [G loss: 0.723133]
315 [D loss: 0.724140, acc.: 40.62%] [G loss: 0.718283]
316 [D loss: 0.701022, acc.: 46.88%] [G loss: 0.693714]
317 [D loss: 0.669312, acc.: 54.69%] [G loss: 0.712866]
318 [D loss: 0.650661, acc.: 50.00%] [G loss: 0.740536]
319 [D loss: 0.673658, acc.: 53.12%] [G loss: 0.

...

7345 [D loss: 0.646583, acc.: 60.94%] [G loss: 1.122774]
7346 [D loss: 0.602084, acc.: 67.19%] [G loss: 1.114005]
7347 [D loss: 0.558159, acc.: 60.94%] [G loss: 1.183835]
7348 [D loss: 0.579607, acc.: 70.31%] [G loss: 0.927719]
7349 [D loss: 0.561931, acc.: 64.06%] [G loss: 1.075351]
7350 [D loss: 0.593886, acc.: 67.19%] [G loss: 1.153818]
7351 [D loss: 0.588577, acc.: 68.75%] [G loss: 1.156796]
7352 [D loss: 0.497953, acc.: 78.12%] [G loss: 1.211213]
7353 [D loss: 0.551153, acc.: 76.56%] [G loss: 1.134137]
7354 [D loss: 0.552426, acc.: 75.00%] [G loss: 1.062894]
7355 [D loss: 0.584964, acc.: 71.88%] [G loss: 1.129664]
7356 [D loss: 0.595182, acc.: 67.19%] [G loss: 1.146875]
7357 [D loss: 0.558208, acc.: 71.88%] [G loss: 1.191553]
7358 [D loss: 0.531929, acc.: 76.56%] [G loss: 1.207739]
7359 [D loss: 0.532807, acc.: 75.00%] [G loss: 1.162770]
7360 [D loss: 0.596942, acc.: 70.31%] [G loss: 0.934293]
7361 [D loss: 0.574929, acc.: 73.44%] [G loss: 1.045355]
7362 [D loss: 0.549597, acc.: 7

7491 [D loss: 0.574670, acc.: 68.75%] [G loss: 1.043774]
7492 [D loss: 0.496486, acc.: 79.69%] [G loss: 1.220099]
7493 [D loss: 0.575787, acc.: 71.88%] [G loss: 1.168941]
7494 [D loss: 0.528574, acc.: 73.44%] [G loss: 1.080495]
7495 [D loss: 0.535126, acc.: 76.56%] [G loss: 1.019804]
7496 [D loss: 0.559153, acc.: 70.31%] [G loss: 1.066842]
7497 [D loss: 0.638468, acc.: 64.06%] [G loss: 1.099184]
7498 [D loss: 0.523735, acc.: 70.31%] [G loss: 1.225084]
7499 [D loss: 0.562278, acc.: 70.31%] [G loss: 1.186860]
7500 [D loss: 0.671726, acc.: 60.94%] [G loss: 1.014679]
7501 [D loss: 0.517964, acc.: 78.12%] [G loss: 1.071288]
7502 [D loss: 0.550339, acc.: 78.12%] [G loss: 1.201947]
7503 [D loss: 0.540188, acc.: 76.56%] [G loss: 1.284016]
7504 [D loss: 0.541180, acc.: 68.75%] [G loss: 1.122263]
7505 [D loss: 0.646459, acc.: 70.31%] [G loss: 1.097668]
7506 [D loss: 0.621906, acc.: 70.31%] [G loss: 0.967463]
7507 [D loss: 0.498123, acc.: 81.25%] [G loss: 0.961119]
7508 [D loss: 0.656597, acc.: 6

7637 [D loss: 0.562526, acc.: 70.31%] [G loss: 1.202058]
7638 [D loss: 0.537618, acc.: 75.00%] [G loss: 1.264630]
7639 [D loss: 0.574610, acc.: 71.88%] [G loss: 1.065514]
7640 [D loss: 0.594802, acc.: 70.31%] [G loss: 1.107817]
7641 [D loss: 0.531705, acc.: 76.56%] [G loss: 1.094439]
7642 [D loss: 0.523553, acc.: 76.56%] [G loss: 1.186173]
7643 [D loss: 0.591673, acc.: 65.62%] [G loss: 1.188315]
7644 [D loss: 0.501929, acc.: 76.56%] [G loss: 1.185805]
7645 [D loss: 0.603650, acc.: 64.06%] [G loss: 1.081286]
7646 [D loss: 0.547926, acc.: 76.56%] [G loss: 1.040579]
7647 [D loss: 0.597758, acc.: 70.31%] [G loss: 1.103990]
7648 [D loss: 0.542632, acc.: 68.75%] [G loss: 1.143484]
7649 [D loss: 0.558052, acc.: 76.56%] [G loss: 1.250464]
7650 [D loss: 0.575994, acc.: 79.69%] [G loss: 1.233501]
7651 [D loss: 0.556140, acc.: 64.06%] [G loss: 1.239704]
7652 [D loss: 0.574075, acc.: 71.88%] [G loss: 1.028507]
7653 [D loss: 0.573997, acc.: 70.31%] [G loss: 1.090255]
7654 [D loss: 0.489057, acc.: 8

7781 [D loss: 0.555747, acc.: 73.44%] [G loss: 1.146318]
7782 [D loss: 0.549156, acc.: 76.56%] [G loss: 1.155655]
7783 [D loss: 0.561298, acc.: 71.88%] [G loss: 1.127558]
7784 [D loss: 0.664907, acc.: 53.12%] [G loss: 1.048824]
7785 [D loss: 0.536358, acc.: 70.31%] [G loss: 1.164522]
7786 [D loss: 0.599713, acc.: 73.44%] [G loss: 1.153387]
7787 [D loss: 0.519554, acc.: 79.69%] [G loss: 1.026563]
7788 [D loss: 0.555037, acc.: 68.75%] [G loss: 1.168500]
7789 [D loss: 0.552458, acc.: 68.75%] [G loss: 1.326844]
7790 [D loss: 0.537488, acc.: 76.56%] [G loss: 1.026021]
7791 [D loss: 0.662758, acc.: 64.06%] [G loss: 1.063357]
7792 [D loss: 0.556242, acc.: 70.31%] [G loss: 1.040304]
7793 [D loss: 0.509125, acc.: 79.69%] [G loss: 1.124400]
7794 [D loss: 0.606072, acc.: 70.31%] [G loss: 0.943828]
7795 [D loss: 0.504141, acc.: 76.56%] [G loss: 0.992527]
7796 [D loss: 0.712325, acc.: 48.44%] [G loss: 1.030250]
7797 [D loss: 0.647178, acc.: 65.62%] [G loss: 1.112035]
7798 [D loss: 0.585321, acc.: 6

7927 [D loss: 0.502666, acc.: 81.25%] [G loss: 1.156627]
7928 [D loss: 0.602536, acc.: 65.62%] [G loss: 1.236807]
7929 [D loss: 0.667788, acc.: 54.69%] [G loss: 1.039065]
7930 [D loss: 0.560251, acc.: 71.88%] [G loss: 1.093769]
7931 [D loss: 0.534922, acc.: 73.44%] [G loss: 1.214497]
7932 [D loss: 0.532942, acc.: 73.44%] [G loss: 1.154849]
7933 [D loss: 0.603456, acc.: 62.50%] [G loss: 1.052150]
7934 [D loss: 0.590310, acc.: 70.31%] [G loss: 1.060827]
7935 [D loss: 0.516160, acc.: 79.69%] [G loss: 1.074493]
7936 [D loss: 0.547883, acc.: 78.12%] [G loss: 1.034586]
7937 [D loss: 0.570828, acc.: 73.44%] [G loss: 1.234981]
7938 [D loss: 0.555500, acc.: 64.06%] [G loss: 1.208443]
7939 [D loss: 0.520455, acc.: 82.81%] [G loss: 1.093212]
7940 [D loss: 0.565257, acc.: 65.62%] [G loss: 0.986788]
7941 [D loss: 0.508529, acc.: 78.12%] [G loss: 1.205486]
7942 [D loss: 0.533998, acc.: 78.12%] [G loss: 0.993737]
7943 [D loss: 0.573513, acc.: 76.56%] [G loss: 0.974984]
7944 [D loss: 0.505661, acc.: 7